## Ligação BD:

In [1]:
import requests  
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import mysql.connector as SQLC
import mysql.connector
from datetime import date
import time
from bs4 import BeautifulSoup
from Bio.SeqFeature import CompoundLocation
from datetime import datetime

In [2]:
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True 
Cursor = DataBase.cursor()

num= input("Escolha o tamanho máximo das seq: ")
Seq_and_length=[]
just_seq=[]
try:
    sql_Seq_length= f"select Gene.ID_genebank, Gene.length from Gene where Gene.length <{num}"
    Cursor.execute(sql_Seq_length)
    for row in Cursor:
        Seq_and_length.append(str(row)) 
    
    sql_Seq= f"select Gene.ID_genebank from Gene where Gene.length <{num}"
    Cursor.execute(sql_Seq)
    for row in Cursor:
        just_seq.append(str(row))
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
if just_seq == []:
        print()
        print('nenhuma seq inferior a esse tamanho, pesquise de novo.')

Escolha o tamanho máximo das seq: 50


In [3]:
#escolher as duas seq e criar listas com output necessários para extrair as seqs
print (Seq_and_length)
list_seq=[]
choose= input('escolha a primeira seq com que quer trabalhar: ')
inter= int(choose)
extr= ', '.join(just_seq)
h= extr.replace("(",'')
hh= h.replace(",)",'')
ID_al= hh.split(', ')
select= ID_al[inter]
list_seq.append(select)

choose2= input('escolha a segunda seq com que quer trabalhar: ')
inter2= int(choose2)
extr2= ', '.join(just_seq)
h2= extr2.replace("(",'')
hh2= h2.replace(",)",'')
ID_al2= hh2.split(', ')
select2= ID_al2[inter2]
list_seq.append(select2)

limpar_aspas_all= ', '.join(ID_al)
retirar_all= limpar_aspas_all.replace("'","")
otput_final_all= retirar_all.split(', ')

limpar_aspas= ', '.join(list_seq)
retirar= limpar_aspas.replace("'","")
otput_final= retirar.split(', ')


#cria lista com todos as seq com o tamanho máximo definido:
todas_as_seq=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= otput_final_all
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    seqs= (info.seq)
    todas_as_seq.append(seqs)
    
    
#cria lista só com as duas seq escolhidas para o alinhamento 
seq_for_al=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist1= otput_final[0]
idlist2= otput_final[1]
handle1 = Entrez.efetch(db=database, id=idlist1, rettype="gb") 
records1 = SeqIO.read(handle1,"gb")
handle1.close()
s1_=records1.seq
handle2 = Entrez.efetch(db=database, id=idlist2, rettype="gb") 
records2 = SeqIO.read(handle2,"gb")
handle2.close()
s2_=records2.seq

print(s1_)
print(s2_)

["('AAADK0043071.1', 21)", "('AAADS0003348.1', 20)", "('AAADW0009410.1', 21)", "('AAADX0043451.1', 21)", "('ABAAM0346030.1', 20)"]
escolha a primeira seq com que quer trabalhar: 1
escolha a segunda seq com que quer trabalhar: 2


/Users/utilizador/opt/anaconda3/lib/python3.9/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


CAACACGGGAAACCTCACCC
GGGATGGAAGATGGCGGCCTA


### Matriz:

In [4]:
import io

class Mat:
    
    def __init__(self, rows, cols):
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    def __repr__(self):
        return str(self)
    def __getitem__ (self, n):
        return self.mat[n]

### Matriz Blossum:

In [119]:
# Lê a matriz blosum 62 de um ficheiro na diretoria

def create_blossum62():
    mat = """
       A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *
    A  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4 
    R -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4 
    N -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4 
    D -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4 
    C  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4 
    Q -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4 
    E -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4 
    G  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4 
    H -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4 
    I -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3 -3 -1 -4 
    L -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4 -3 -1 -4 
    K -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0  1 -1 -4 
    M -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3 -1 -1 -4 
    F -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1 -3 -3 -1 -4 
    P -1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2 -2 -1 -2 -4 
    S  1 -1  1  0 -1  0  0  0 -1 -2 -2  0 -1 -2 -1  4  1 -3 -2 -2  0  0  0 -4 
    T  0 -1  0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1  1  5 -2 -2  0 -1 -1  0 -4 
    W -3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1  1 -4 -3 -2 11  2 -3 -4 -3 -2 -4 
    Y -2 -2 -2 -3 -2 -1 -2 -3  2 -1 -1 -2 -1  3 -3 -2 -2  2  7 -1 -3 -2 -1 -4 
    V  0 -3 -3 -3 -1 -2 -2 -3 -3  3  1 -2  1 -1 -2 -2  0 -3 -1  4 -3 -2 -1 -4 
    B -2 -1  3  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3  4  1 -1 -4 
    Z -1  0  0  1 -3  3  4 -2  0 -3 -3  1 -1 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4 
    X  0 -1 -1 -1 -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -2  0  0 -2 -1 -1 -1 -1 -1 -4 
    * -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  1"""

    D = {}

    fields, *M = [L.strip().split() for L in mat.splitlines() if L.strip() != '']

    for L in M:
        k, *L = L
        for f, v in zip(fields, L):
            if f not in D:
                D[f] = {}
            D[f][k] = int(v)
    return D

sm = create_blossum62()

# score of a position (column) 
def score_pos (c1, c2, sm, gap_penalty):
    if c1 == "-" or c2=="-":
        return gap_penalty
    else:
        return sm[c1][c2]

### NW:

In [132]:
class NW:
    def __init__(self, s1, s2, gap_penalty, match_score, mismatch_penalty):
        self.s1 = s1 
        self.s2 = s2 
        self.gap_penalty= gap_penalty
        self.match_score= match_score
        self.mismatch_penalty= mismatch_penalty
        self.mat = Mat(len(s1) + 1, len(s2) + 1)
        self.tr  = Mat(len(s1) + 1, len(s2) + 1)

        for L in range(len(s1)):
            self.mat[L + 1][0] = self.gap_penalty * (L + 1) 
            self.tr[L + 1][0]  = 'C' 
        for C in range(len(s2)):
            self.mat[0][C + 1] = self.gap_penalty * (C + 1) 
            self.tr[0][C + 1]  = 'E'

        for L, x1 in enumerate(s1):
            for C, x2 in enumerate(s2):
                possiveis = [
                    self.mat[L  ][C    ] + (self.match_score if x1 == x2 else self.mismatch_penalty),   # Diagonal
                    self.mat[L+1][C    ] + self.gap_penalty,               # Esquerda
                    self.mat[L  ][C + 1] + self.gap_penalty,               # Cima
                ]
                dirs = "DEC"
                self.mat[L + 1][C + 1] = max(possiveis)
                self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]

                
    def rebuild(self):
        L = len(self.s1)
        C = len(self.s2)
        S1 = ""
        S2 = ""
    
        dirs = {
            'D' : (-1, -1),
            'E' : ( 0, -1),
            'C' : (-1,  0)
        }
        while L > 0 or C > 0:
            
            DL, DC = dirs[self.tr[L][C]]
            if self.tr[L][C] == "D":
                S1 = self.s1[L - 1] + S1 
                S2 = self.s2[C - 1] + S2
            elif self.tr[L][C] == "E":
                S1 = '-' + S1
                S2 = self.s2[C - 1] + S2
            else:
                S1 = self.s1[L - 1] + S1
                S2 = '-' + S2        
    
            L += DL
            C += DC
    

        return S1, S2 
    
    
    def __repr__(self): 
        cols = "-" + self.s2
        lins = "-" + self.s1
        with io.StringIO("") as S:
            print(' ', *cols, sep = '   ', file = S)  
            for L, linha in zip(lins, self.mat):
                print(L, *[f'{x:3d}' for x in linha], file = S)
            print(file = S)

            print(' ', *cols, file = S) 
            for L, linha in zip(lins, self.tr):
                print(L, *linha, file = S)

            return S.getvalue()
        

In [114]:
import unittest

class TestNW(unittest.TestCase):
    def test_init(self):
        nw = NW("AGCT", "GATT", -1, 1, -1)
        self.assertEqual(nw.s1, "AGCT")
        self.assertEqual(nw.s2, "GATT")
        self.assertEqual(nw.gap_penalty, -1)
        self.assertEqual(nw.match_score, 1)
        self.assertEqual(nw.mismatch_penalty, -1)

    def test_matrix_construction(self):
        nw = NW("AGTACGCA", "TACG", -1, 1, -1)
        self.assertEqual(nw.mat[1][0], -1)
        self.assertEqual(nw.mat[1][1], -1)
        self.assertEqual(nw.mat[2][2], -1)
        self.assertEqual(nw.tr[1][0], 'C')
        self.assertEqual(nw.tr[3][2], 'E')
        self.assertEqual(nw.tr[6][4], 'D')

    def test_rebuild(self):
        nw = NW("AGTACGCA", "TACG", -1, 1, -1)
        s1, s2 = nw.rebuild()
        self.assertEqual(s1, "AGTACGCA")
        self.assertEqual(s2, "--TACG--")

        
    def test_score(self):
        nw = NW("AGTACGCA", "TACG", -1, 1, -1)
        score = nw.mat[len(nw.s1)][len(nw.s2)]
        self.assertEqual(score, 0)

suite = unittest.TestLoader().loadTestsFromTestCase(TestNW)
unittest.TextTestRunner(verbosity=3).run(suite)


test_init (__main__.TestNW) ... ok
test_matrix_construction (__main__.TestNW) ... ok
test_rebuild (__main__.TestNW) ... ok
test_score (__main__.TestNW) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

### SW:

In [133]:
class SW:
    def __init__(self, s1, s2, gap_penalty, match_score, mismatch_penalty):
        self.s1 = s1 
        self.s2 = s2 
        self.gap_penalty= gap_penalty
        self.match_score= match_score
        self.mismatch_penalty= mismatch_penalty
        self.score = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
        self.tr = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
       

        for L in range(len(s1)): 
            self.tr[L + 1][0]  = 'E' 
        for C in range(len(s2)):
            self.tr[0][C + 1]  = 'C'
                
        for i in range(1, len(self.s1) + 1):
            for j in range(1, len(self.s2) + 1):
                match = self.score[i - 1][j - 1] + (self.match_score if self.s1[i - 1] == self.s2[j - 1] else self.mismatch_penalty)
                L = self.score[i - 1][j] + self.gap_penalty
                U = self.score[i][j - 1] + self.gap_penalty
                self.score[i][j] = max(0, match, L, U)
                possiveis= (match, L, U,0)
                dirs = 'DEC0'
                self.tr[i][j] = dirs[possiveis.index(self.score[i][j])]
                
                    
    def rebuild(self):
        L = len(self.s1)
        C = len(self.s2)
            
        max_i, max_j = 0, 0
        max_score = 0
        for i in range(1, L + 1):
            for j in range(1, C + 1):
                if self.score[i][j] > max_score:
                    max_i, max_j = i, j
                    max_score = self.score[i][j]
        i, j = max_i, max_j
        alignD1= "" 
        alignD2= ""
        while self.score[i][j] != 0:
            current_score = self.score[i][j]
            diagonal_score = self.score[i - 1][j - 1]
            up_score = self.score[i][j - 1]
            left_score = self.score[i - 1][j]

            if current_score == diagonal_score + (self.match_score if self.s1[i - 1] == self.s2[j - 1] else self.mismatch_penalty):
                alignD1 += self.s1[i - 1]
                alignD2 += self.s2[j - 1]
                i -= 1
                j -= 1
            elif current_score == left_score + self.gap_penalty:
                alignD1 += self.s1[i - 1]
                alignD2 += "-"
                i -= 1
            elif current_score == up_score + self.gap_penalty:
                alignD1 += "-"
                alignD2 += self.s2[j - 1]
                j -= 1
        
                
        return alignD1[::-1], alignD2[::-1] 
    
    
    
    def __repr__(self): 
        cols = "-" + self.s2
        lins = "-" + self.s1
        with io.StringIO("") as S:
            print(' ', *cols, sep = '   ', file = S)  
            for L, linha in zip(lins, self.score):
                print(L, *[f'{x:3d}' for x in linha], file = S)
            print(file = S)

            print(' ', *cols, file = S) 
            for L, linha in zip(lins, self.tr ):
                print(L, *linha, file = S)

            return S.getvalue()



In [116]:
class TestSW(unittest.TestCase):
    def test_init(self):
        sw = SW("AGCT", "GATT", -1, 1, -1)
        self.assertEqual(sw.s1, "AGCT")
        self.assertEqual(sw.s2, "GATT")
        self.assertEqual(sw.gap_penalty, -1)
        self.assertEqual(sw.match_score, 1)
        self.assertEqual(sw.mismatch_penalty, -1)

    def test_matrix_construction(self):
        sw = SW("AGTACGCA", "TACG", -1, 1, -1)
        self.assertEqual(sw.score[1][0], 0)
        self.assertEqual(sw.score[1][1], 0)
        self.assertEqual(sw.score[7][4], 3)
        self.assertEqual(sw.tr[1][2], 'D')
        self.assertEqual(sw.tr[5][4], 'C')
        self.assertEqual(sw.tr[7][4], 'E')
        self.assertEqual(sw.tr[1][1], '0')

    def test_rebuild(self):
        sw = SW("AGTACGCA", "TACG", -1, 1, -1)
        s1, s2 = sw.rebuild()
        self.assertEqual(s1, "TACG")
        self.assertEqual(s2, "TACG")

        
    def test_max_score(self):
        sw = SW("AGTACGCA", "TACG", -1, 1, -1)
        self.assertEqual(sw.score[6][4],4)
 
suite = unittest.TestLoader().loadTestsFromTestCase(TestSW)
unittest.TextTestRunner(verbosity=3).run(suite)


test_init (__main__.TestSW) ... ok
test_matrix_construction (__main__.TestSW) ... ok
test_max_score (__main__.TestSW) ... ok
test_rebuild (__main__.TestSW) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.006s

OK


6 4


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [134]:
class NW_blossum:
    def __init__(self, s1, s2, sm, gap_penalty):
        self.s1 = s1 
        self.s2 = s2 
        self.sm= sm
        self.gap_penalty= gap_penalty
        self.mat = Mat(len(s1) + 1, len(s2) + 1)
        self.tr  = Mat(len(s1) + 1, len(s2) + 1)

        for L in range(len(s1)):
            self.mat[L + 1][0] = self.gap_penalty * (L + 1) 
            self.tr[L + 1][0]  = 'C' 
        for C in range(len(s2)):
            self.mat[0][C + 1] = self.gap_penalty * (C + 1) 
            self.tr[0][C + 1]  = 'E'

        for L, x1 in enumerate(s1):
            for C, x2 in enumerate(s2):
                possiveis = [
                    self.mat[L  ][C    ] + score_pos (self.s1[L-1], self.s2[C-1], self.sm, self.gap_penalty),   # Diagonal
                    self.mat[L+1][C    ] + self.gap_penalty,               # Esquerda
                    self.mat[L  ][C + 1] + self.gap_penalty,               # Cima
                ]
                dirs = "DEC"
                self.mat[L + 1][C + 1] = max(possiveis)
                self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]
                
    def rebuild(self):
        L = len(self.s1)
        C = len(self.s2)
        S1 = ""
        S2 = ""
    
        dirs = {
            'D' : (-1, -1),
            'E' : ( 0, -1),
            'C' : (-1,  0)
        }
        while L > 0 or C > 0:
            
            DL, DC = dirs[self.tr[L][C]]
            if self.tr[L][C] == "D":
                S1 = self.s1[L - 1] + S1 
                S2 = self.s2[C - 1] + S2
            elif self.tr[L][C] == "E":
                S1 = '-' + S1
                S2 = self.s2[C - 1] + S2
            else:
                S1 = self.s1[L - 1] + S1
                S2 = '-' + S2        
    
            L += DL
            C += DC

        return S1, S2 
    def __repr__(self): 
        cols = "-" + self.s2
        lins = "-" + self.s1
        with io.StringIO("") as S:
            print(' ', *cols, sep = '   ', file = S)  
            for L, linha in zip(lins, self.mat):
                print(L, *[f'{x:3d}' for x in linha], file = S)
            print(file = S)

            print(' ', *cols, file = S) 
            for L, linha in zip(lins, self.tr):
                print(L, *linha, file = S)

            return S.getvalue()
        

In [138]:
class TestNW_blossum(unittest.TestCase):
    def test_init(self):
        nwb = NW_blossum("AGTACGCA", "TACG", sm, -1)
        self.assertEqual(nwb.s1, "AGTACGCA")
        self.assertEqual(nwb.s2, "TACG")
        self.assertEqual(nwb.sm, sm)
        self.assertEqual(nwb.gap_penalty, -1)
        
    def test_matrix_construction(self):
        nwb = NW_blossum("AGTACGCA", "TACG", sm, -1)
        self.assertEqual(nwb.mat[1][0], -1)
        self.assertEqual(nwb.mat[1][1], 0)
        self.assertEqual(nwb.mat[5][2], 8)
        self.assertEqual(nwb.tr[1][0], 'C')
        self.assertEqual(nwb.tr[3][2], 'E')
        self.assertEqual(nwb.tr[6][4], 'D')

    def test_rebuild(self):
        nwb = NW_blossum("AGTACGCA", "TACG", sm, -1)
        s1, s2 = nwb.rebuild()
        self.assertEqual(s1, "AGTACGCA")
        self.assertEqual(s2, "--TAC--G")

        
    def test_score(self):
        nwb = NW_blossum("AGTACGCA", "TACG", sm, -1)
        score = nwb.mat[len(nwb.s1)][len(nwb.s2)]
        self.assertEqual(score, 20)

suite = unittest.TestLoader().loadTestsFromTestCase(TestNW_blossum)
unittest.TextTestRunner(verbosity=3).run(suite)

test_init (__main__.TestNW_blossum) ... ok
test_matrix_construction (__main__.TestNW_blossum) ... ok
test_rebuild (__main__.TestNW_blossum) ... ok
test_score (__main__.TestNW_blossum) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.009s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [140]:
class SW_blossum:
    def __init__(self, s1, s2, sm, gap_penalty):
        self.s1 = s1 
        self.s2 = s2 
        self.sm= sm
        self.gap_penalty= gap_penalty
        self.score = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
        self.tr = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
       

        for L in range(len(s1)): 
            self.tr[L + 1][0]  = 'E' 
        for C in range(len(s2)):
            self.tr[0][C + 1]  = 'C'
                
        for i in range(1, len(self.s1) + 1):
            for j in range(1, len(self.s2) + 1):
                match = self.score[i - 1][j - 1] + score_pos (self.s1[i-1], self.s2[j-1], self.sm, self.gap_penalty)
                L = self.score[i - 1][j] + self.gap_penalty
                U = self.score[i][j - 1] + self.gap_penalty
                self.score[i][j] = max(0, match, L, U)
                possiveis= (match, L, U,0)
                dirs = 'DEC0'
                self.tr[i][j] = dirs[possiveis.index(self.score[i][j])]
                
                
    def rebuild(self):
    
        L = len(self.s1)
        C = len(self.s2)
            
        max_i, max_j = 0, 0
        max_score = 0
        for i in range(1, L + 1):
            for j in range(1, C + 1):
                if self.score[i][j] > max_score:
                    max_i, max_j = i, j
                    max_score = self.score[i][j]
        i, j = max_i, max_j
        alignD1= "" 
        alignD2= ""
        while self.score[i][j] != 0:
            current_score = self.score[i][j]
            diagonal_score = self.score[i - 1][j - 1]
            up_score = self.score[i][j - 1]
            left_score = self.score[i - 1][j]

            if current_score == diagonal_score + score_pos (self.s1[i-1], self.s2[j-1], self.sm, self.gap_penalty):
                alignD1 += self.s1[i - 1]
                alignD2 += self.s2[j - 1]
                i -= 1
                j -= 1
            elif current_score == left_score + self.gap_penalty:
                alignD1 += self.s1[i - 1]
                alignD2 += "-"
                i -= 1
            elif current_score == up_score + self.gap_penalty:
                alignD1 += "-"
                alignD2 += self.s2[j - 1]
                j -= 1
        
                
        return alignD1[::-1], alignD2[::-1] 
    
    
    
    def __repr__(self): 
        cols = "-" + self.s2
        lins = "-" + self.s1
        with io.StringIO("") as S:
            print(' ', *cols, sep = '   ', file = S)  
            for L, linha in zip(lins, self.score):
                print(L, *[f'{x:3d}' for x in linha], file = S)
            print(file = S)

            print(' ', *cols, file = S) 
            for L, linha in zip(lins, self.tr ):
                print(L, *linha, file = S)

            return S.getvalue()



In [150]:
class TestSW_blossum(unittest.TestCase):
    def test_init(self):
        swb = SW_blossum("AGTACGCA", "TACG",sm, -1)
        self.assertEqual(swb.s1, "AGTACGCA")
        self.assertEqual(swb.s2, "TACG")
        self.assertEqual(swb.sm, sm)
        self.assertEqual(swb.gap_penalty, -1)

    def test_matrix_construction(self):
        swb = SW_blossum("AGTACGCA", "TACG",sm, -1)
        self.assertEqual(swb.score[1][0], 0)
        self.assertEqual(swb.score[3][4], 8)
        self.assertEqual(swb.score[7][4], 23)
        self.assertEqual(swb.tr[1][2], 'D')
        self.assertEqual(swb.tr[5][4], 'C')
        self.assertEqual(swb.tr[7][4], 'E')
        self.assertEqual(swb.tr[2][1], '0')

    def test_rebuild(self):
        swb = SW_blossum("AGTACGCA", "TACG",sm, -1)
        s1, s2 = swb.rebuild()
        self.assertEqual(s1, "TACG")
        self.assertEqual(s2, "TACG")

        
    def test_max_score(self):
        swb = SW_blossum("AGTACGCA", "TACG",sm, -1)
        self.assertEqual(swb.score[6][4],24)
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestSW_blossum)
unittest.TextTestRunner(verbosity=3).run(suite)
        

test_init (__main__.TestSW_blossum) ... ok
test_matrix_construction (__main__.TestSW_blossum) ... ok
test_max_score (__main__.TestSW_blossum) ... ok
test_rebuild (__main__.TestSW_blossum) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [ ]:
escolha_de_input= input('Prefere utilizar blossum ou definir o input(1- você define o input, 2- utiliza blossum): ')

def_map = {"1": NW, "2": NW_blossum}
try:
    escolja_cls = def_map[escolha_de_input]
    if escolja_cls == NW:
        es_gap= int(input('escolha o valor de gap: '))
        es_match= int(input('escolha o valor de match: '))
        es_mis=int(input('escolha o valor de mismatch: '))
        alin= escolja_cls(s1_,s2_, es_gap ,es_match ,es_mis)
        print(alin)
        print()
        print(*alin.rebuild(), sep="\n")
        
    else:
        es_gap= int(input('escolha o valor de gap: '))
        align= NW_blossum(s1_,s2_,sm,-4)
        print(align)
        print(*align.rebuild(), sep="\n")
except KeyError:
    raise ValueError("Tipo de alinhamento inválido.")

### Multiple align:

In [86]:
def consensus(s1_, s2_):
    res = ""
    for x, y in zip(s1_, s2_):
        if x == y:
            res += x
        elif x == '-':
            res += y
        else:
            res += x
    return res

escolha_alinh= input('escolha o tipo de alinhamento que quer(SW ou NW): ')
al_map = {"SW": SW, "NW": NW}

try:
    alinhamento_cls = alg_map[escolha_alinh]
except KeyError:
    raise ValueError("Tipo de alinhamento inválido.")
                     
es_gap= int(input('escolha o valor de gap: '))
es_match= int(input('escolha o valor de match: '))
es_mis=int(input('escolha o valor de mismatch: '))
print()
print (Seq_and_length)
seq3= int(input('escolha a seq da BD que pretende adicionar ao alinhamento: '))
print()
s3= todas_as_seq[seq3]            
a1, a2 = alinhamento_cls(s1_,s2_,es_gap,es_match,es_mis).rebuild()
print(a1)
print(a2)
print()
m1 = consensus(a1,a2)
print(m1)
print()
m2, a3 = alinhamento_cls(m1, s3, es_gap,es_match,es_mis).rebuild()
print(m2,a3, sep="\n")
print()


escolha o tipo de alinhamento que quer(SW ou NW): SW
escolha o valor de gap: -4
escolha o valor de match: 3
escolha o valor de mismatch: -1

["('AAADK0043071.1', 21)", "('AAADS0003348.1', 20)", "('AAADW0009410.1', 21)", "('AAADX0043451.1', 21)", "('ABAAM0346030.1', 20)"]
escolha a seq da BD que pretende adicionar ao alinhamento: 4

ACGGGAAACCTCACCC
ATGGAAGATGGCGGCC

ACGGGAAACCTCACCC

GGGAAACCTCACCC
GGTGAACTTCTGCC



## Enzimas de corte:

In [104]:
import re
enzimas = {"R" : "[GA]", "Y" : "[CT]", "M" : "[AC]", "K" : "[GT]", "S" : "[GC]", "W" : "[AT]", "B" : "[^A]", "D" : "[^C]", "H" : "[^G]", "V" : "[^T]", "N" : "[ACGT]"}

print (enzimas)
print()
enzima = input('selecione o que deseja cortar: ')
print()

print (Seq_and_length)
seq_enzimas= int(input('escolha o id da BD que pretende analisar: '))
s_enzimas= todas_as_seq[seq_enzimas] 

def converte(enz):
    return ''.join([enzimas.get(x, x) for x in enz])

def enz_corte(enzima):
    e1, e2 = enzima.split('^')
    return f"(?<={converte(e1)}){converte(e2)}"

pos = [m.span()[0] for m in re.finditer(enz_corte(enzima), str(s_enzimas))]

a=[str(s_enzimas[i:j]) for i, j in zip([0, *pos], [*pos, len(str(s_enzimas))])]
print()
print(a)
    

    

{'R': '[GA]', 'Y': '[CT]', 'M': '[AC]', 'K': '[GT]', 'S': '[GC]', 'W': '[AT]', 'B': '[^A]', 'D': '[^C]', 'H': '[^G]', 'V': '[^T]', 'N': '[ACGT]'}

selecione o que deseja cortar: R^K

["('AAADK0043071.1', 21)", "('AAADS0003348.1', 20)", "('AAADW0009410.1', 21)", "('AAADX0043451.1', 21)", "('ABAAM0346030.1', 20)"]
escolha o id da BD que pretende analisar: 4
['G', 'TTTG', 'G', 'TGAACTTCTGCCTA']


In [100]:
print (str(s1_))

GGGATGGAAGATGGCGGCCTA
